# Experimentation results

## Using filtering system with 10 augmented and 1 real images processed through GPT Vision

## Pascal Results

In [34]:
import os
import pandas as pd
import glob

def process_csv(file_path):
    df = pd.read_csv(file_path)
    filtered_df = df[(df['metric'] == 'Accuracy') & (df['split'] == 'Validation')]
    best_results = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
    best_results['filename'] = os.path.basename(file_path)
    return best_results

# Get all CSV files with 'pascal-0-1' in the name
csv_files = glob.glob('test-researchers-pascal/*-0-1*.csv')

# Process all files and concatenate results
all_results = pd.concat([process_csv(file) for file in csv_files], ignore_index=True)

# Reorder columns to have filename first
all_results = all_results[['filename', 'examples_per_class', 'value']]

# Sort by value (accuracy) in descending order
all_results_sorted = all_results.sort_values('value', ascending=False)

print("All results:")
display(all_results_sorted)

All results:


,filename,examples_per_class,value
4,results-0-1-aug.csv,1,0.662918
2,my-pascal-0-1-aug-v1.csv,1,0.661101
3,my-pascal-0-1-no-aug-v2.csv,1,0.660327
5,my-pascal-0-1-no-aug-v1.csv,1,0.659004
0,my-pascal-0-1-aug-v2.csv,1,0.656475
1,results-0-1-no-aug.csv,1,0.649085


In [40]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    filtered_df = df[(df['metric'] == 'Accuracy') & (df['split'] == 'Validation')]
    best_results = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
    best_results['filename'] = os.path.basename(file_path)
    return best_results

# Get all CSV files with 'pascal-0-1' in the name
csv_files = glob.glob('test-researchers-pascal/*-0-2*.csv')

# Process all files and concatenate results
all_results = pd.concat([process_csv(file) for file in csv_files], ignore_index=True)

# Reorder columns to have filename first
all_results = all_results[['filename', 'examples_per_class', 'value']]

# Sort by value (accuracy) in descending order
all_results_sorted = all_results.sort_values('value', ascending=False)

print("All results:")
display(all_results_sorted)

All results:


,filename,examples_per_class,value
5,results-0-2-aug.csv,2,0.711716
3,results-0-2-no-aug.csv,2,0.691173
1,my-pascal-0-2-no-aug-v1.csv,2,0.688558
0,pascal-0-2-no-aug-v2.csv,2,0.685982
4,results-0-2-aug.csv,1,0.662918
2,results-0-2-no-aug.csv,1,0.649085


In [38]:
0.691173 - 0.688558

0.002615000000000034

In [41]:
0.691173 - 0.685982

0.005191000000000057

### Old prompts to OpenAI

In [ ]:
# prompt = f"""Analyze these images of class {class_name}. The first image is real and serves as reference.
# For each of the remaining 10 augmented images, compare them to the real one and provide float scores from 0 to 1 for:
# Index: from 1 to 10 in order of the augmented images
# a) Quality: Overall visual fidelity and clarity compared to the real one.
# b) Realism: How well it matches real-world expectations set by the reference image.
# c) Relevance: How well it represents a {class_name} compared to the reference image.
# d) Detail Preservation: Retention of important class-specific features.

# Respond ONLY with a list of JSON objects, one for each non-reference image, in this format:
# [
#   {{"index": 1, "quality": 0.0, "realism": 0.0, "relevance": 0.0, "detail_preservation": 0.0, "explanation": "Brief explanation."}}
# ]
# """

In [ ]:
prompt = f"""You are evaluating augmented images for an image classification task. The first image of class {class_name} is real and serves as reference.
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

Index: from 1 to 10 in order of the augmented images

1. Semantic Similarity: How well does the image maintain the core semantic content of the original class? (0 = completely different, 1 = identical)

2. Feature Preservation: Are key features that define the class still present and recognizable? (0 = no key features preserved, 1 = all key features preserved)

3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum beneficial diversity)

4. Visual Quality: Is the image clear, well-formed, and free from obvious artifacts? (0 = poor quality, 1 = excellent quality)

5. Realism: Does the image look natural and plausible for the given class? (0 = completely unrealistic, 1 = indistinguishable from a real photo)

Provide your response ONLY as a list of JSON objects one for each non-reference augmented image, in this format:

[
  {{"index": 1, "semantic_similarity": 0.0, "feature_preservation": 0.0, "diversity": 0.0, "visual_quality": 0.0, "realism": 0.0, "explanation": "Brief explanation focusing on changes and their impact on classification."}}
]
"""

In [ ]:
prompt = f"""You are evaluating augmented images for an image classification task. The first image of class {class_name} is real and serves as reference.
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:
Index: from 1 to 10 in order of the augmented images
1. Semantic Similarity: How well does the image maintain the core semantic content of the original class? (0 = completely different, 1 = identical)
2. Feature Preservation: Are key features that define the class still present and recognizable? (0 = no key features preserved, 1 = all key features preserved)
3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum beneficial diversity)
4. Visual Quality: Is the image clear, well-formed, and free from obvious artifacts? (0 = poor quality, 1 = excellent quality)
5. Realism: Does the image look natural and plausible for the given class? (0 = completely unrealistic, 1 = indistinguishable from a real photo)
6. Contextual Consistency: How well does the image maintain the appropriate context or environment for the class? (0 = completely inconsistent, 1 = perfectly consistent)
7. Pose/Viewpoint Variation: How well does the image provide a useful variation in pose or viewpoint while remaining identifiable? (0 = no variation, 1 = optimal variation)
8. Lighting/Color Variation: How effectively does the image introduce meaningful changes in lighting or color that could aid in generalization? (0 = no variation, 1 = optimal variation)
9. Class Distinguishability: How easily distinguishable is this class from other potentially similar classes in this augmented version? (0 = easily confused with other classes, 1 = highly distinguishable)
Provide your response ONLY as a list of JSON objects one for each non-reference augmented image, in this format:
[
  {{"index": 1, "semantic_similarity": 0.0, "feature_preservation": 0.0, "diversity": 0.0, "visual_quality": 0.0, "realism": 0.0, "contextual_consistency": 0.0, "pose_viewpoint_variation": 0.0, "lighting_color_variation": 0.0, "class_distinguishability": 0.0, "explanation": "Brief explanation focusing on changes and their impact on classification."}}
]
"""

## claude prompt v0

In [ ]:
prompt = f"""You are an expert computer vision model tasked with evaluating the quality of augmented images for training a robust image classifier. The first image of class {{class_name}} is real and serves as a reference. For each of the remaining 10 augmented images, evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

1. Diversity: How different is this augmentation from the original image in terms of visual and semantic attributes? (0 = identical, 1 = maximum beneficial diversity)

2. Realism: How realistic and plausible does the augmented image look? (0 = completely unrealistic, 1 = indistinguishable from a real image)

3. Feature Preservation: How well does the augmentation maintain the critical features that define the class of the original image? (0 = critical features lost, 1 = all critical features perfectly preserved)

4. Contextual Variation: How well does the augmentation introduce new contexts or backgrounds while maintaining relevance? (0 = no contextual change, 1 = highly relevant new context)

5. Robustness Potential: How likely is this augmentation to help the model learn invariance to irrelevant transformations or resistance to noise? (0 = unlikely to improve robustness, 1 = highly likely to improve robustness)

6. Edge Case Representation: Does this augmentation represent an edge case or rare scenario that could improve the model's generalization? (0 = common scenario, 1 = highly valuable edge case)

7. Overall Quality: Considering all factors, how valuable is this augmentation for training a classifier? (0 = detrimental to training, 1 = extremely valuable)

8. Class Consistency: How well does the augmented image maintain the same class as the original image? (0 = class completely changed, 1 = class perfectly maintained)

9. Artifact-Free: To what extent is the augmented image free from unnatural artifacts or distortions that could negatively impact training? (0 = severe artifacts present, 1 = completely artifact-free)

10. Unique Contribution: How much does this augmentation contribute unique aspects not present in other augmentations? (0 = redundant with other augmentations, 1 = highly unique contribution)

Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:

[
  {{
    "index": 1,
    "diversity": 0.0,
    "realism": 0.0,
    "feature_preservation": 0.0,
    "contextual_variation": 0.0,
    "robustness_potential": 0.0,
    "edge_case_representation": 0.0,
    "overall_quality": 0.0,
    "class_consistency": 0.0,
    "artifact_free": 0.0,
    "unique_contribution": 0.0,
    "explanation": "Brief explanation of how changes affect classification and quality, including any improvement suggestions"
  }}
]

After evaluating all images, provide a brief summary (max 3 sentences) of the overall quality of the augmentation set, highlighting strengths and areas for improvement.
"""

## claude prompt v1

In [ ]:
prompt = f"""You are evaluating augmented images for an image classification task. The first image of class {class_name} is real and serves as reference.
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:
Index: from 1 to 10 in order of the augmented images
1. Class Representation: How well does the image maintain the core semantic content, key features, and distinguishability of the original class? (0 = completely irrelevant or indistinguishable, 1 = perfectly relevant, feature-complete, and highly distinguishable)
2. Visual Fidelity: Is the image clear, well-formed, free from artifacts, natural-looking, and in an appropriate context for the given class? (0 = poor quality, unrealistic, or out of context, 1 = excellent quality, realistic, and in perfect context)
3. Structural Integrity: How well does the image maintain the overall structure and proportions expected for the class? (0 = severely distorted structure, 1 = perfect structural integrity)
4. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum beneficial diversity)
5. Beneficial Variations: How effectively does the image introduce meaningful changes in pose, viewpoint, lighting, or color that could aid in generalization? (0 = no variation, 1 = optimal variation)
Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:
[
{{
"index": 1,
"class_representation": 0.0,
"visual_fidelity": 0.0,
"structural_integrity": 0.0,
"diversity": 0.0,
"beneficial_variations": 0.0,
"explanation": "Brief explanation of how changes affect classification and quality"
}}
]
"""

## claude prompt v2

In [ ]:
prompt = f"""You are an expert computer vision model evaluating augmented images for an image classification task. The first image of class {{class_name}} is real and serves as a reference. For each of the remaining 10 augmented images, evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

1. Class Representation: How well does the image maintain the core semantic content, key features, and distinguishability of the original class? (0 = completely irrelevant or indistinguishable, 1 = perfectly relevant, feature-complete, and highly distinguishable)

2. Visual Fidelity: Is the image clear, well-formed, free from artifacts, natural-looking, and in an appropriate context for the given class? (0 = poor quality, unrealistic, or out of context, 1 = excellent quality, realistic, and in perfect context)

3. Structural Integrity: How well does the image maintain the overall structure and proportions expected for the class? (0 = severely distorted structure, 1 = perfect structural integrity)

4. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum beneficial diversity)

5. Beneficial Variations: How effectively does the image introduce meaningful changes in pose, viewpoint, lighting, or color that could aid in generalization? (0 = no variation, 1 = optimal variation)

6. Edge Case Representation: Does this augmentation represent a valuable edge case or rare scenario that could improve the model's generalization? (0 = common scenario, 1 = highly valuable edge case)

7. Robustness Potential: How likely is this augmentation to improve the model's robustness to irrelevant transformations or noise? (0 = unlikely to improve robustness, 1 = highly likely to improve robustness)

Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:

[
  {{
    "index": 1,
    "class_representation": 0.0,
    "visual_fidelity": 0.0,
    "structural_integrity": 0.0,
    "diversity": 0.0,
    "beneficial_variations": 0.0,
    "edge_case_representation": 0.0,
    "robustness_potential": 0.0,
    "explanation": "Brief explanation of how changes affect classification and quality, including any unique aspects or improvement suggestions"
  }}
]

## Claude prompt v3

In [ ]:
prompt = f"""You are an expert computer vision model evaluating augmented images for an image classification task. The first image of class {{class_name}} is real and serves as a reference. For each of the remaining 10 augmented images, evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

1. Class Representation: How well does the image maintain the core semantic content, key features, and distinguishability of the original class? (0 = completely irrelevant or indistinguishable, 1 = perfectly relevant, feature-complete, and highly distinguishable)

2. Visual Quality: How well does the augmented image maintain a clear, well-formed, and natural appearance while preserving the expected structure and proportions for the given class? (0 = poor quality, unrealistic, or severely distorted structure, 1 = excellent quality, realistic, and perfect structural integrity)

3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum beneficial diversity)

4. Beneficial Variations: How effectively does the image introduce meaningful changes in pose, viewpoint, lighting, or color that could aid in generalization? (0 = no variation, 1 = optimal variation)

Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:

[
  {{
    "index": 1,
    "class_representation": 0.0,
    "visual_quality": 0.0,
    "diversity": 0.0,
    "beneficial_variations": 0.0,
    "explanation": "Brief explanation of how changes affect classification and quality"
  }}
]
"""

## OpenAI prompt v1

In [ ]:
prompt = f"""
You are evaluating augmented images for an image classification task. The first image of class {class_name} is real and serves as the reference. 
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

Index: from 1 to 10 in order of the augmented images
1. Visual Fidelity: Is the image clear, free of artifacts, realistic, and visually consistent with what is expected for the class? (0 = poor quality, 1 = excellent quality)
2. Class Identity: How well does the image maintain the core features and distinguishability from other classes? (0 = completely irrelevant, 1 = perfectly relevant)
3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum useful diversity)
4. Beneficial Variations: How well does the image provide useful variations in pose, viewpoint, lighting, or color that could aid in generalization while maintaining class identity? (0 = no variation, 1 = optimal variation)
5. Structural Consistency: How well does the image maintain appropriate context, structure, and proportions for the class? (0 = highly distorted, 1 = perfectly consistent)

Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:
[
{{
 "index": 1,
 "visual_fidelity": 0.0,
 "class_identity": 0.0,
 "diversity": 0.0,
 "beneficial_variations": 0.0,
 "structural_consistency": 0.0,
 "explanation": "Brief explanation of how changes affect classification and quality based on the above criteria."
}}
]
"""


## V2

In [ ]:
prompt = f"""
You are one of the world's best image quality experts. You're critical and detail-oriented. For every image you rate correctly, you get paid $100,000. You value your reputation and always make the most objective assessment possible
You are evaluating augmented images for an image classification task. The first image of class {class_name} is real and serves as the reference. 
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:

Index: from 1 to 10 in order of the augmented images
1. Visual Fidelity: Is the image clear, free of artifacts, realistic, and visually consistent with what is expected for the class? (0 = poor quality, 1 = excellent quality)
2. Class Identity: How well does the image maintain the core features and distinguishability from other classes? (0 = completely irrelevant, 1 = perfectly relevant)
3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum useful diversity)
4. Beneficial Variations: How well does the image provide useful variations in pose, viewpoint, lighting, or color while maintaining class identity? (0 = no variation, 1 = optimal variation)
5. Structural Consistency: How well does the image maintain appropriate context, structure, and proportions for the class? (0 = highly distorted, 1 = perfectly consistent)

Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:
[
{{
 "index": 1,
 "visual_fidelity": 0.0,
 "class_identity": 0.0,
 "diversity": 0.0,
 "beneficial_variations": 0.0,
 "structural_consistency": 0.0,
 "explanation": "Brief explanation of how changes affect classification and quality."
}}
]
"""

## V3

In [ ]:
prompt = f"""
You are evaluating augmented images for an image classification task. The first image of class {{class_name}} is real and serves as the reference. 
For each of the remaining 10 augmented images, please evaluate them based on the following criteria, providing a score between 0.0 and 1.0 for each:
Index: from 1 to 10 in order of the augmented images
1. Visual and Structural Quality: Is the image clear, free of artifacts, realistic, and does it maintain appropriate context, structure, and proportions for the class? (0 = poor quality, highly distorted, 1 = excellent quality, perfectly consistent)
2. Class Identity: How well does the image maintain the core features and distinguishability from other classes? (0 = completely irrelevant, 1 = perfectly relevant)
3. Diversity: How different is the image from the reference while still maintaining class identity? (0 = identical, 1 = maximum useful diversity)
4. Beneficial Variations: How well does the image provide useful variations in pose, viewpoint, lighting, or color that could aid in generalization while maintaining class identity? (0 = no variation, 1 = optimal variation)
Provide your response as a list of JSON objects, one for each non-reference augmented image, in this format:
[
{{
 "index": 1,
 "visual_and_structural_quality": 0.0,
 "class_identity": 0.0,
 "diversity": 0.0,
 "beneficial_variations": 0.0,
 "explanation": "Brief explanation of how changes affect classification and quality based on the above criteria."
}}
]
"""

### Current transform

In [ ]:
# if is_train:
#     return transforms.Compose([
#         transforms.Resize(self.image_size),
#         transforms.ToTensor(),
#         transforms.ConvertImageDtype(torch.float),
#         transforms.Lambda(lambda x: x.expand(3, *self.image_size)),
#         transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
#     ])

### Results for pascal torch dataset v1

In [42]:
import os
import pandas as pd
import glob

def process_csv(file_path):
    df = pd.read_csv(file_path)
    filtered_df = df[(df['metric'] == 'Accuracy') & (df['split'] == 'Validation')]
    best_results = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
    best_results['filename'] = os.path.basename(file_path)
    return best_results

# Get all CSV files with 'pascal-0-1' in the name
csv_files = glob.glob('my-results/*-0-1*.csv')

# Process all files and concatenate results
all_results = pd.concat([process_csv(file) for file in csv_files], ignore_index=True)

# Reorder columns to have filename first
all_results = all_results[['filename', 'examples_per_class', 'value']]

# Sort by value (accuracy) in descending order
all_results_sorted = all_results.sort_values('value', ascending=False)

print("All results:")
display(all_results_sorted)

# Get the best overall strategy and score
best_strategy = all_results_sorted.iloc[0]
print("\nBest strategy:")
print(f"Filename: {best_strategy['filename']}")
print(f"Examples per class: {best_strategy['examples_per_class']}")
print(f"Best accuracy: {best_strategy['value']:.4f}")

All results:


,filename,examples_per_class,value
24,results-filtered-pascal-0-1-top_n_overall-0_90...,1,0.673252
25,results-filtered-pascal-0-1-creative_percentil...,1,0.670356
15,results-filtered-pascal-0-1-percentile_overall...,1,0.669661
0,results-filtered-pascal-0-1-creative_top_n_ove...,1,0.669074
27,results-filtered-pascal-0-1-percentile_overall...,1,0.668760
19,results-filtered-pascal-0-1-top_n_overall_weig...,1,0.668760
3,filtered-pascal-0-1-percentile_overall_weighte...,1,0.668579
9,filtered-pascal-0-1-top_n_overall_weighted-0_9...,1,0.668579
32,filtered-pascal-0-1-creative_percentile_overal...,1,0.662556
10,results-filtered-pascal-0-1-percentile_class_w...,1,0.662546



Best strategy:
Filename: results-filtered-pascal-0-1-top_n_overall-0_90.csv
Examples per class: 1
Best accuracy: 0.6733


In [47]:
all_results_sorted['filename'].to_list()

['results-filtered-pascal-0-1-top_n_overall-0_90.csv',
 'results-filtered-pascal-0-1-creative_percentile_overall-0_10.csv',
 'results-filtered-pascal-0-1-percentile_overall-0_10.csv',
 'results-filtered-pascal-0-1-creative_top_n_overall-0_90.csv',
 'results-filtered-pascal-0-1-percentile_overall_weighted-0_10.csv',
 'results-filtered-pascal-0-1-top_n_overall_weighted-0_90.csv',
 'filtered-pascal-0-1-percentile_overall_weighted-0_10_v2.csv',
 'filtered-pascal-0-1-top_n_overall_weighted-0_90_v2.csv',
 'filtered-pascal-0-1-creative_percentile_overall-0_10_v2.csv',
 'results-filtered-pascal-0-1-percentile_class_weighted-0_10.csv',
 'results-filtered-pascal-0-1-creative_percentile_class-0_10.csv',
 'results-pascal-0-1.csv',
 'results-1-pascal-0-1-transform2.csv',
 'results-filtered-pascal-0-1-combined_overall_weighted-NA.csv',
 'results-filtered-pascal-0-1-top_n_class-0_90.csv',
 'filtered-pascal-0-1-combined_overall_weighted-NA_v2.csv',
 'results-filtered-pascal-0-1-creative_top_n_class-0_

In [ ]:
all_results_sorted

### Results from researchers with next train transform:

In [94]:
# train_transform = transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.RandomHorizontalFlip(p=0.5),
#             transforms.RandomRotation(degrees=15.0),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [1]:
import pandas as pd
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their_v1 = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their_v1

FileNotFoundError: [Errno 2] No such file or directory: 'my_results/results_0_1.csv'

In [18]:
!pwd

/Users/andrew/Thesis/smart-image-augmentation/results


In [19]:
!ls

comparing-our-accuracy-and-researchers.ipynb
experiments.ipynb
filtration-results
my-results
replicating-paper-results.ipynb
researchers-results


In [20]:
!ls my-results/

results-filtered-pascal-0-1-combined-0_10-0_90.csv
results-filtered-pascal-0-1-combined-0_20-0_80.csv
results-filtered-pascal-0-1-combined-0_30-0_70.csv
results-filtered-pascal-0-1-percentile-0_10.csv
results-filtered-pascal-0-1-percentile-0_20.csv
results-filtered-pascal-0-1-percentile-0_30.csv
results-filtered-pascal-0-1-percentile-0_40.csv
results-filtered-pascal-0-1-percentile-0_50.csv
results-filtered-pascal-0-1-top_n-0_50.csv
results-filtered-pascal-0-1-top_n-0_60.csv
results-filtered-pascal-0-1-top_n-0_70.csv
results-filtered-pascal-0-1-top_n-0_80.csv
results-filtered-pascal-0-1-top_n-0_90.csv
results-filtered-pascal-0-1-top_n-0_90_v1.csv
results-filtered-pascal-0-1-top_n-0_90_v2.csv
results-pascal-0-1_v1.csv
results-pascal-0-1_v2.csv


### Results from researchers with next train transform:

In [97]:
# transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [113]:
import pandas as pd
their_df = pd.read_csv('my_results/results_0_1_v2.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their_v2 = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their_v2

,examples_per_class,value
0,1,0.649085


In [17]:
df = pd.read_csv('my-results/results-filtered-pascal-0-1-top_n-0_90_v1.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659246


In [15]:
df = pd.read_csv('my-results/results-filtered-pascal-0-1-top_n-0_90_v2.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.655907


### My implementation results based on their code with next train transform:

In [106]:
# train_transform = transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.RandomHorizontalFlip(p=0.5),
#             transforms.RandomRotation(degrees=15.0),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [134]:
my_df = pd.read_csv('results_1_pascal-0-1_5.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v1 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v1

,examples_per_class,value
0,1,0.661101


In [121]:
best_results_my_v1.value.to_numpy()

array([0.66110075])

In [126]:
print(f"Difference between our and researchers implementation with augmentation transform: {best_results_their_v1.value.to_numpy()[0]-best_results_my_v1.value.to_numpy()[0]}")


Difference between our and researchers implementation with augmentation transform: 0.0018177000000000332


,examples_per_class,value
0,1,0.655907


### My implementation results based on their code with next train transform:

In [108]:
# transforms.Compose([
#                 transforms.Resize(self.image_size),
#                 transforms.ToTensor(),
#                 transforms.ConvertImageDtype(torch.float),
#                 transforms.Lambda(lambda x: x.expand(3, *self.image_size)),
#                 transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
#             ])

In [124]:
my_df = pd.read_csv('results_1_pascal-0-1_4.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.659004


In [127]:
print(f"Difference between our and researchers implementation with simple transform: {best_results_their_v2.value.to_numpy()[0]-best_results_my_v2.value.to_numpy()[0]}")


Difference between our and researchers implementation with simple transform: -0.009919289999999914


In [136]:
my_df = pd.read_csv('results_1_pascal-0-1_7.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.661101


In [139]:
my_df = pd.read_csv('results_1_pascal-0-1_9.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [143]:
my_df = pd.read_csv('results_1_pascal-0-1_10.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [3]:
import pandas as pd
my_df = pd.read_csv('my_results/results_1_pascal-0-1_11.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [4]:
import pandas as pd
my_df = pd.read_csv('my_results/results_1_pascal-0-1_12.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [5]:
import pandas as pd
my_df = pd.read_csv('my_results/results_1_pascal-0-1_13.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.660327


In [9]:
!ls

comparing-our-accuracy-and-researchers.ipynb
experiments.ipynb
filtration-results
my-results
replicating-paper-results.ipynb
researchers-results


In [11]:
import pandas as pd
my_df = pd.read_csv('my-results/results-pascal-0-1_v1.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.659004


In [12]:
import pandas as pd
my_df = pd.read_csv('my-results/results-pascal-0-1_v2.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.660327


In [70]:
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918


In [ ]:
df = pd.read_csv('my-results/results-filtered-pascal-0-1-top_n-0_90_v1.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.655907


In [82]:
df = pd.read_csv('results_1_pascal-0-1_2.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


In [84]:
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918


## with simple transform

In [83]:
df = pd.read_csv('results_1_pascal-0-1_3.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


In [89]:
0.662918 - 0.659004 

0.003913999999999973

In [92]:
0.662918 - 0.661101

0.0018169999999999575

In [85]:
df = pd.read_csv('results_1_pascal-0-1_4.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


## With flips and rotations transform

In [87]:
df = pd.read_csv('results_1_pascal-0-1_5.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.661101


In [91]:
their_df = pd.read_csv('results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.649085


In [90]:
0.662918 - 0.661101

0.0018169999999999575

In [50]:
df = pd.read_csv('results_1_pascal-0-1_aws.csv')
df

,seed,examples_per_class,epoch,value,metric,split
0,0,1,0,1.978202,Loss,Training
1,0,1,0,2.326135,Loss,Validation
2,0,1,0,0.850066,Accuracy,Training
3,0,1,0,0.500000,Accuracy,Validation
4,0,1,0,1.988846,Loss Airplane,Training
...,...,...,...,...,...,...
4195,0,1,49,1.000000,Accuracy Train,Validation
4196,0,1,49,0.000453,Loss Television,Training
4197,0,1,49,0.364092,Loss Television,Validation
4198,0,1,49,1.000000,Accuracy Television,Training


In [53]:
their_df = pd.read_csv('my_results/results_0_1.csv')
their_df

,Unnamed: 0,seed,examples_per_class,epoch,value,metric,split
0,0,0,1,0,2.044756,Loss,Training
1,1,0,1,0,2.356342,Loss,Validation
2,2,0,1,0,0.827288,Accuracy,Training
3,3,0,1,0,0.536613,Accuracy,Validation
4,4,0,1,0,1.903856,Loss Airplane,Training
...,...,...,...,...,...,...,...
4195,4195,0,1,49,0.852861,Accuracy Train,Validation
4196,4196,0,1,49,0.000490,Loss Television,Training
4197,4197,0,1,49,0.491666,Loss Television,Validation
4198,4198,0,1,49,1.000000,Accuracy Television,Training


In [24]:
their_df = pd.read_csv('results_0_2.csv')

filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918
1,2,0.711716


### Results from my implementation with top_k=10 detail=low

In [26]:
df = pd.read_csv('results_1_pascal-0-1.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.65


In [32]:
df = pd.read_csv('results_1_filtered-pascal-0-1-percentile-0_10.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.7


,seed,examples_per_class,epoch,value,metric,split
3,0,1,0,0.45,Accuracy,Validation
87,0,1,1,0.45,Accuracy,Validation
171,0,1,2,0.55,Accuracy,Validation
255,0,1,3,0.60,Accuracy,Validation
339,0,1,4,0.60,Accuracy,Validation
423,0,1,5,0.60,Accuracy,Validation
507,0,1,6,0.60,Accuracy,Validation
591,0,1,7,0.60,Accuracy,Validation
675,0,1,8,0.65,Accuracy,Validation
759,0,1,9,0.65,Accuracy,Validation


In [33]:
df = pd.read_csv('results_1_filtered-pascal-0-1-percentile-0_10_collab.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.7


### Results from my implementation with top_k=9 detail=low

In [16]:
df = pd.read_csv('my_results/results_1_percentile_0_1.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.75


In [17]:
filtered_df

,seed,examples_per_class,epoch,value,metric,split
3,0,1,0,0.60,Accuracy,Validation
87,0,1,1,0.75,Accuracy,Validation
171,0,1,2,0.75,Accuracy,Validation
255,0,1,3,0.75,Accuracy,Validation
339,0,1,4,0.75,Accuracy,Validation
423,0,1,5,0.75,Accuracy,Validation
507,0,1,6,0.75,Accuracy,Validation
591,0,1,7,0.75,Accuracy,Validation
675,0,1,8,0.75,Accuracy,Validation
759,0,1,9,0.75,Accuracy,Validation


## Other

In [13]:
import pandas as pd

# Load your dataset from the CSV file
# Update this to the correct file path
df = pd.read_csv('updated_augmented_images_with_class.csv')

# Calculate mean, median, and 75th percentile of overall_score
mean_score = df['overall_score'].mean()
median_score = df['overall_score'].median()
percentile_75_score = df['overall_score'].quantile(0.75)
percentile_25_score = df['overall_score'].quantile(0.25)

# Filter the data based on the calculated thresholds
mean_threshold_images = df[df['overall_score'] >= mean_score]
median_threshold_images = df[df['overall_score'] >= median_score]
percentile_75_threshold_images = df[df['overall_score'] >= percentile_75_score]
percentile_25_threshold_images = df[df['overall_score'] >= percentile_25_score]

# Count total images and images per class for each threshold
mean_threshold_counts = mean_threshold_images['class'].value_counts()
median_threshold_counts = median_threshold_images['class'].value_counts()
percentile_75_threshold_counts = percentile_75_threshold_images['class'].value_counts()
percentile_25_threshold_counts = percentile_25_threshold_images['class'].value_counts()

# Print the results
print("Mean Threshold:")
print(f"Total images above mean: {mean_threshold_images.shape[0]}")
print(f"Images per class above mean:\n{mean_threshold_counts}")

print("\nMedian Threshold:")
print(f"Total images above median: {median_threshold_images.shape[0]}")
print(f"Images per class above median:\n{median_threshold_counts}")

print("\n75th Percentile Threshold:")
print(f"Total images above 75th percentile: {percentile_75_threshold_images.shape[0]}")
print(f"Images per class above 75th percentile:\n{percentile_75_threshold_counts}")

print("\n25th Percentile Threshold:")
print(f"Total images above 75th percentile: {percentile_25_threshold_images.shape[0]}")
print(f"Images per class above 75th percentile:\n{percentile_25_threshold_counts}")


Mean Threshold:
Total images above mean: 90
Images per class above mean:
class
cow             10
boat            10
dog             10
potted plant    10
motorcycle       9
car              8
bicycle          6
bus              5
bird             4
sheep            4
sofa             4
chair            3
dining table     3
bottle           3
cat              1
Name: count, dtype: int64

Median Threshold:
Total images above median: 102
Images per class above median:
class
cow             10
boat            10
motorcycle      10
dog             10
potted plant    10
bicycle         10
car              8
bus              6
sheep            5
bird             4
sofa             4
airplane         4
chair            3
dining table     3
bottle           3
cat              2
Name: count, dtype: int64

75th Percentile Threshold:
Total images above 75th percentile: 53
Images per class above 75th percentile:
class
boat            10
car              8
motorcycle       8
potted plant     7
cow 